<p style="float:right"> <img src="assets/orange.png" alt="Orange logo" width="40" /> <img src="assets/ulb.jpg" alt="ULB logo" width="40" /> <img src="assets/mlg.png" alt="MLG logo" width="160" /> <img src="assets/innoviris.jpg" alt="Innoviris logo" width="200" /></p>

**_Notebook for the project Machu-Picchu written by Théo Verhelst_**<br/>
_Supervisors at Orange: Denis Mercier, Jeevan Shrestha_<br/>
_Academic supervision: Gianluca Bontempi (ULB MLG)_
# Estimation of the probability of counterfactuals from simulated data

In [ ]:
library(ggplot2)
source("estimate_cf.R")
source("utils.R")

In [ ]:
n_runs <- 5000
n_samples_space <- list(min = 10, max = 500)
n_approx_space <- list(min = 2, max = 50)
n_cores <- 4
use_churn_convention <- TRUE
A_space <- list(min = 0.1, max = 15)

params <- sample_dirichlet(N=n_runs, alpha=c(1, 1, 1, 1))
# The commented lines are there to either randomize the parameters
# or use a constant value.
alpha_o <- 0.947
beta_o  <- 0.020
gamma_o <- 0.017
delta_o <- 0.017
params <- data.frame(
    #alpha = params[,1],
    #beta  = params[,2],
    #gamma = params[,3],
    #delta = params[,4],
    alpha = alpha_o,
    beta  = beta_o,
    gamma = gamma_o,
    delta = delta_o,
    #n_samples = as.integer(runif(min=n_samples_space$min, max=n_samples_space$max, n=n_runs)),
    n_samples = 2000,
    #n_approx = as.integer(runif(min=n_approx_space$min, max=n_approx_space$max, n=n_runs)),
    n_approx = 50,
    A = runif(min=A_space$min, max=A_space$max, n=n_runs)
    #A = 10^runif(min=-1.5, max=1, n=n_runs)
    #A = 10
)

params$phi <- params$alpha * params$delta + params$beta * params$gamma

start_time <- Sys.time()
simulation_results <- do.call(rbind, lapply(1:nrow(params), function(i) {
    # Determine the parameters of this simulation run
    alpha <- params[i,]$alpha
    beta  <- params[i,]$beta
    gamma <- params[i,]$gamma
    delta <- params[i,]$delta
    n_samples <- params[i,]$n_samples
    n_approx <- params[i,]$n_approx
    A <- params[i,]$A

    U <- sample_dirichlet(N=n_samples, alpha=c(alpha, beta, gamma, delta) * A)
    data <- data.frame(
        alpha = U[,1],
        beta = U[,2],
        gamma = U[,3],
        delta = U[,4]
    )
    data$phi <- data$alpha * data$delta - data$beta * data$gamma
    data$S_0 <- data$beta + data$delta
    data$S_1 <- data$gamma + data$delta
    data$uplift <- ifelse(use_churn_convention, data$S_0 - data$S_1, data$S_1 - data$S_0)
    
    model_variance <- (mean(data$S_0 * (1 - data$S_0) / n_approx) + mean(data$S_1 * (1 - data$S_1) / n_approx)) / 2 
    expected_entropy <- entropy_dirichlet(c(alpha, beta, gamma, delta) * A)
    
    # The estimated score \hat S_i is based on a hypothetical set
    # of i.i.d samples from Y_i of size n_approx. Hence no normal
    # approximation is needed to have a noisy version of S_i
    data$S_0_hat <- suppressWarnings(rbinom(nrow(data), n_approx, data$S_0) / n_approx)
    data$S_0_hat[data$S_0 == 1] <- 1
    data$S_0_hat[data$S_0 == 0] <- 0
    data$S_1_hat <- suppressWarnings(rbinom(nrow(data), n_approx, data$S_1) / n_approx)
    data$S_1_hat[data$S_1 == 1] <- 1
    data$S_1_hat[data$S_1 == 0] <- 0
    data$uplift_hat <- ifelse(use_churn_convention, data$S_0_hat - data$S_1_hat, data$S_1_hat - data$S_0_hat)
    
    statistics <- estimate_from_simulation(data)

    elapsed_time <- Sys.time() - start_time
    remaining_time <- (nrow(params) * elapsed_time / i) - elapsed_time
    rem <- as.numeric(remaining_time, units="secs")
    cat(sprintf(
        "\r%d / %d, %d min, %d sec remaining          ",
        i,
        nrow(params),
        rem %/% 60,
        floor(rem %% 60)
    ))
    flush.console()


    statistics <- c(
        statistics,
        list(
            alpha_param = alpha,
            beta_param  = beta,
            gamma_param = gamma,
            delta_param = delta,
            cov_alpha_delta = cov(data$alpha, data$delta),
            cov_beta_gamma = cov(data$beta, data$gamma),
            cov_01 = cov(data$S_0, data$S_1),
            n_approx = n_approx,
            n_samples = n_samples,
            A = A,
            var_uplift_hat = mean((data$uplift - data$uplift_hat)^2),
            model_variance = model_variance,
            expected_entropy = expected_entropy
        )
    )

    return(as.data.frame(statistics))
}))

saveRDS(simulation_results, file = "runs/simulation_results.rds")

In [ ]:
simulation_results <- readRDS("runs/simulation_results.rds")

#### Plot the bounds and estimators as a function of the true value

In [ ]:
plot_width <- 15
plot_height <- 8
plot_units <- "cm"
font_size <- 11
font_family <- "Times"
expr_entropy <- expression(H(paste(list(Y[0]^(i),Y[1]^(i)), " ", symbol("\x7C"), " ", list(alpha^(i), beta^(i), gamma^(i), delta^(i)))))
expr_variance <- expression(paste("Model variance ", Var ~ bgroup("(", widehat(S)[t]^(i), ")")))

In [ ]:
n_bins <- 10
to_plot <- data.frame(bin_num = 1:n_bins)
to_plot$bin <- levels(cut(runif(100), breaks=(0:n_bins)/n_bins))

for (var in c("alpha", "beta", "gamma", "delta")) {
  bins <- cut(simulation_results[,var], (0:n_bins)/n_bins)
  for (var_suffix in c("", "_ind", "_lb_s_ind", "_ub_s_ind", "_lb_s_pop", "_ub_s_pop")) {
    varname <- paste0(var, var_suffix)
    to_plot[,paste0("mean_", varname)] <- 0
    for (l in levels(bins)) {
      to_plot[to_plot$bin == l, paste0("mean_", varname)] <- mean(simulation_results[bins == l, varname], na.rm=TRUE)
    }
  }
}

var <- "delta"

whisker_width <- 0.2
to_plot$bin_left <- as.numeric(to_plot$bin_num) - whisker_width
to_plot$bin_right <- as.numeric(to_plot$bin_num) + whisker_width

p <- ggplot(to_plot, aes(x = bin)) +
  geom_point(
    aes_(y = as.name(paste0("mean_", var, "_ind")), shape="est_val"),
    position = position_nudge(x = 0.5)
  ) +
  geom_point(
    aes_(y = as.name(paste0("mean_", var)), shape="true_val"),
    position = position_nudge(x = 0.5)
  )

for (var_suffix in c("lb_s_ind", "ub_s_ind", "lb_s_pop", "ub_s_pop")) {
  p <- p + geom_segment(
    aes_(
      x = as.name("bin_left"),
      xend = as.name("bin_right"),
      y = as.name(paste0("mean_", var, "_", var_suffix)),
      yend = as.name(paste0("mean_", var, "_", var_suffix)),
      linetype = substr(var_suffix, 6, 9)
    ),
    position = position_nudge(x = 0.5),
    color = "#333333"
  )
}
p <- p + scale_y_continuous(n.breaks = n_bins) +
  scale_x_discrete(labels = as.character((0:n_bins) / n_bins)) +
  scale_shape_manual(
    name   = "Legend",
    values = c(true_val = 1, est_val = 4),
    labels = c(true_val = "True value", est_val = "Estimand")
  ) +
  scale_linetype_manual(
    name   = NULL,
    values = c(ind = 1, pop = 2),
    labels = c(ind = "Uplift bounds", pop = "Fréchet bounds")
  ) +
 labs(x = paste0("True ", var, " (stratified)"), y = NULL) + 
  theme_bw() +
  theme(
    legend.box = "vertical",
    legend.position = "right",
    text = element_text(size = font_size, family = font_family)
  )
#ggsave(paste0("pdf/", var, "_bounds.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
s <- simulation_results
cat("Uplift bounds width      = ", 100 * mean(s$beta_ub_s_ind - s$beta_lb_s_ind, na.rm=T), " +- ", 100 * sd(s$beta_ub_s_ind - s$beta_lb_s_ind, na.rm=T), "\n")
cat("Fréchet bounds width = ", 100 * mean(s$beta_ub_s_pop - s$beta_lb_s_pop, na.rm=T), " +- ", 100 * sd(s$beta_ub_s_pop - s$beta_lb_s_pop, na.rm=T), "\n")
cat("Uplift point RMSE          = ", 100 * RMSE(s$beta_param, s$beta_ind), "\n")
cat("Fréchet mid-point RMSE = ", 100 * RMSE((s$beta_lb_s_pop + s$beta_ub_s_pop) / 2, s$beta_param), "\n")
cat("Uplift mid-point RMSE      = ", 100 * RMSE((s$beta_lb_s_ind + s$beta_ub_s_ind) / 2, s$beta_param), "\n")

In [ ]:
p <- ggplot(simulation_results, aes(x = H_Y0_Y1_X, y = beta_ub_s_ind - beta_lb_s_ind)) +
    geom_point(alpha=0.3) +
    labs(x = expr_entropy, y = "Uplift bounds span") + 
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))

#ggsave(paste0("pdf/entropy_bounds_span.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x = H_Y0_Y1_X, y = beta_ind - beta_param)) +
    geom_point(alpha=0.3) +
    labs(x = expr_entropy, y = "Point estimator bias") + 
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))

#ggsave(paste0("pdf/entropy_bias_napprox_50_nsamples_2000.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x = n_samples, y = beta_ub_s_ind - beta_lb_s_ind)) +
    geom_point(alpha=0.3) +
    labs(x = "Number of samples N", y = "Uplift bounds span") + 
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))

#ggsave(paste0("pdf/n_sample_bounds_span_napprox_50_A_1.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x = n_samples, y = beta_ind - beta_param)) +
    geom_point(alpha=0.3) +
    labs(x = "Number of samples N", y = "Point estimator error") +
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))
#ggsave(paste0("pdf/n_sample_error.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x = model_variance, y = (beta_ub_s_ind - beta_lb_s_ind))) +
    geom_point(alpha=0.3) +
    scale_x_continuous(trans="log10") +
    #geom_hline(yintercept=mean(simulation_results$beta_ub_s_ind_true)-mean(simulation_results$beta_lb_s_ind_true), color="#4444FF") +
    labs(x = expr_variance, y = "Uplift bounds span") +
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))
#ggsave(paste0("pdf/model_variance_bounds_span.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x = model_variance, y = beta_ind - beta)) +
    geom_point(alpha=0.3) +
    scale_x_continuous(trans="log10") +
    labs(x = expr_variance, y = "Point estimator error") +
    #geom_hline(yintercept=mean(simulation_results$phi), color="#4444FF") +
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))
#ggsave(paste0("pdf/model_variance_error.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p

In [ ]:
p <- ggplot(simulation_results, aes(x=phi)) +
    geom_histogram(bins=30) +
    labs(x = expression(E*group("[", phi^(i), "]")), y = "Number of simuations") +
    theme_bw() +
    theme(text = element_text(size = font_size, family = font_family))
#ggsave(paste0("pdf/phi_distribution.pdf"), plot = p, width = plot_width, height = plot_height, units = plot_units)
p